In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
import cirq
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, f1_score
import time

In [6]:

# ===== 1. DATA =====
iris = load_iris()
X = iris.data
y = iris.target

mask = y < 2
X = X[mask]
y = y[mask]

scaler = StandardScaler()
X = scaler.fit_transform(X)[:, :2]  # use first 2 features

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32)

# ===== 2. CIRCUIT HELPERS =====
n_qubits = 2
qubits = cirq.LineQubit.range(n_qubits)

def build_circuit(params, x):
    circuit = cirq.Circuit()
    # Feature encoding
    for i in range(n_qubits):
        circuit.append(cirq.rx(x[i])(qubits[i]))
    # Variational layers
    for i in range(n_qubits):
        circuit.append(cirq.rx(params[i, 0])(qubits[i]))
        circuit.append(cirq.rz(params[i, 1])(qubits[i]))
    # Entanglement
    circuit.append(cirq.CNOT(qubits[0], qubits[1]))
    # Measurement
    circuit.append(cirq.measure(qubits[0], key="m"))
    return circuit

def run_circuit(params, x):
    sim = cirq.Simulator()
    circuit = build_circuit(params, x)
    result = sim.run(circuit, repetitions=100)
    m = np.mean(result.measurements["m"])
    return 1.0 - m  # expectation value mapped to [0,1]

# ===== 3. TORCH + PARAMETER-SHIFT =====
class CirqFunction(torch.autograd.Function):
    @staticmethod
    def forward(ctx, params, x):
        params_np = params.detach().numpy()
        x_np = x.detach().numpy()
        out = run_circuit(params_np, x_np)
        ctx.save_for_backward(params, x)
        return torch.tensor(out, dtype=torch.float32)

    @staticmethod
    def backward(ctx, grad_output):
        params, x = ctx.saved_tensors
        params_np = params.detach().numpy()
        x_np = x.detach().numpy()
        shift = np.pi / 2
        grads = np.zeros_like(params_np)

        for i in range(params_np.shape[0]):
            for j in range(params_np.shape[1]):
                plus = params_np.copy()
                plus[i, j] += shift
                minus = params_np.copy()
                minus[i, j] -= shift
                grad = (run_circuit(plus, x_np) - run_circuit(minus, x_np)) / 2
                grads[i, j] = grad

        return grad_output * torch.tensor(grads, dtype=torch.float32), None

# ===== 4. MODEL =====
class QuantumClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.params = nn.Parameter(0.01 * torch.randn(n_qubits, 2))

    def forward(self, x):
        outputs = []
        for xi in x:
            outputs.append(CirqFunction.apply(self.params, xi))
        return torch.stack(outputs).reshape(-1, 1)


In [ ]:

# ===== 5. TRAINING =====
model = QuantumClassifier()
optimizer = optim.Adam(model.parameters(), lr=0.1)
loss_fn = nn.BCELoss()

epochs = 30
for epoch in range(epochs):
    optimizer.zero_grad()
    y_pred = model(X_train)
    loss = loss_fn(y_pred, y_train.reshape(-1, 1))
    loss.backward()
    optimizer.step()
    if (epoch + 1) % 1 == 0:
        print(f"Epoch {epoch+1} | Loss: {loss.item():.4f}")


Epoch 1 | Loss: 4.8035
Epoch 2 | Loss: 7.2924
Epoch 3 | Loss: 7.2718
Epoch 4 | Loss: 5.9862
Epoch 5 | Loss: 3.4971
Epoch 6 | Loss: 3.3749
Epoch 7 | Loss: 4.5429
Epoch 8 | Loss: 2.1069
Epoch 9 | Loss: 3.2544
Epoch 10 | Loss: 3.2270


In [8]:
# ===== 6. EVALUATION =====
with torch.no_grad():
    start_time = time.perf_counter()
    preds = model(X_test).numpy()
    preds_binary = (preds > 0.5).astype(int)
    end_time = time.perf_counter()

    mse = mean_squared_error(y_test, preds.flatten())
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_test, preds.flatten())
    f1 = f1_score(y_test.numpy().astype(int), preds_binary)

print("\n--- Metrics ---")
print(f"MSE: {mse:.6f}")
print(f"RMSE: {rmse:.6f}")
print(f"MAE: {mae:.6f}")
print(f"F1-score: {f1:.6f}")
print(f"Inference/sample: {(end_time - start_time)/len(X_test):.6f} sec")


--- Metrics ---
MSE: 0.404490
RMSE: 0.635995
MAE: 0.526000
F1-score: 0.538462
Inference/sample: 0.001255 sec
